### Maximize Profit

In [1]:
# Importing Modules

from pyomo.environ import *
import pandas as pd

import random

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_rows", 99)
pd.set_option("display.max_columns", 99)
random.seed(0)

#### Parameters

<!-- 

---------------------------------------------------------------------------------------------------------------------------

df['Customers'] = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',]
df['Log'] = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]
df['Lat'] = [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,]

---------------------------------------------------------------------------------------------------------------------------
  
df['Customers'] = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5']
df['Log'] = [0, 1, 1, -1, -1, 0]
df['Lat'] = [0, 1, 2, 2, 1, 3]


-->

In [5]:
# Number of Customers 
total_customers = 30

# Number of Sellers / Trucks
total_sellers = 3

# Seller capacity
sellers_capacity = 28800 # s (8h/day)

# Truck capacity
truck_capacity = 60 # unit (High Pressure Cylinders)

# Truck Speed
velocity = 13.9 # m/s (Normal speed of a Truck (50 km/h))

# Dutarion Service
change_duration = 300 # 5 minutes on average to change a cylinder
service_duration = 300 # 5 minutes per stop

# ID Source
warehouse = 0
 
# the bigM
bigM = total_customers + 1

# Bidirectional Graph: G(V,A)
# (C[i], C[j]) : Cost

X, Y  = 2600, 2600

cols = ['Customers', 'Log', 'Lat', 'Cyn', 'Val']
df = pd.DataFrame(columns = cols)
df.loc[0, cols] = ['Warehouse', 0, 0, 0, 0]

for c in range(total_customers):

    ind = c + 1    
    df.loc[ind, 'Customers'] = f'C{c}'
    df.loc[ind, 'Log'] = 0 + X * random.random()
    df.loc[ind, 'Lat'] = 0 + Y * random.random()
    df.loc[ind, 'Cyn'] = random.choice(range(1, 10))
    df.loc[ind, 'Val'] = random.choice(range(80, 1500)) * df.loc[ind, 'Cyn']




# Calculate the euclidian distance between each point on the map
cost = {}
for i in df.index:
    for j in df.index:

        x = df.loc[i, 'Log'] - df.loc[j, 'Log']
        y = df.loc[i, 'Lat'] - df.loc[j, 'Lat']
        cost[(i, j)] = (x**2 + y**2)**(0.5)



### mTSP Model


In [ ]:
# Creating the Model
model = ConcreteModel()

# Creating the Sets
model.Customers = RangeSet(0, total_customers) # Set of Clients
model.Sellers = RangeSet(1, total_sellers) # Set of Sallers

# Creating Variables
    
# a[c, s] = 1, if customer c is served by salesman s
model.a = Var(  model.Customers, 
                model.Sellers,
                within = Binary,
                initialize = 0 
            )

# u[i] = node potentials, 
model.u = Var(  model.Customers, 
                within = NonNegativeIntegers,
            )

# x[i, j, s] = 1, if the saller s went from customer i to customer j
model.x = Var(  model.Customers, 
                model.Customers, 
                model.Sellers,
                within = Binary,
                initialize = 0
            )

# time[s] = time worked
model.time = Var(   model.Sellers,
                    within = NonNegativeReals,
                    initialize = 0)

# Objective Function : Maximize Sales Profit
def profit_objective(model):

    profit = sum(   model.a[i, s] * df.loc[i, 'Val']
                    for i in model.Customers 
                    for s in model.Sellers
            )
    
    return profit 


model.objective = Objective(    rule = profit_objective, 
                                sense = maximize
                            )

#### Constraints

In [ ]:
def constraint_time(model, s):
    """
    Defines a time variable for a seller in an optimization model.
    """

   
    service_time = (    sum( model.a[i, s] * df.loc[i, 'Cyn'] * change_duration + model.a[i, s] * service_duration
                        for i in model.Customers if i != warehouse) ) 
    
    walk_time = sum(    model.x[i, j, s] * cost[i,j] / velocity 
                        for i in model.Customers 
                        for j in model.Customers)

    return model.time[s] == service_time + walk_time

model.C0 = Constraint(  model.Sellers, 
                        rule = constraint_time
                    )

In [ ]:
def constraint_truck_capacity(model, s):
    """
    The truck cannor carry more than its capacity.
    """
    
    return sum(model.a[i, s] * df.loc[i, 'Cyn'] for i in model.Customers) <= truck_capacity
     
model.C1 = Constraint(  model.Sellers, 
                        rule = constraint_truck_capacity
                    )

In [ ]:
def constraint_sellers_capacity(model, s):
    ''' 
    The seller cannot serve more than his capacity.
    Each seller has a time capacity to serve customers.
    The seller spends an average of 300s (5m) per cylinder change for each customer, 
    in addition to 600s (10m) for customer service.
    And it moves at an average of 13.9 m/s.
    '''



    return model.time[s] <= sellers_capacity

model.C2 = Constraint(  model.Sellers, 
                        rule = constraint_sellers_capacity
                    )

In [ ]:
def constraint_visitation(model, i):
    '''
    Constraint function that enforces that each customer is assigned to at most one seller.
    '''
    if i != warehouse:
        return sum(model.a[i, s] for s in model.Sellers) <= 1
    else:
        return Constraint.Skip

model.C3 = Constraint(  model.Customers,
                        rule = constraint_visitation  )

In [ ]:
def constraint_sellers_out(model, s):
    ''' 
    Ensures that each seller (s) has an outbound sub route from the warehouse.
    '''
    return sum( model.x[warehouse, j, s] 
                for j in model.Customers if (cost[warehouse, j] > 0)) == 1

model.C4 = Constraint(  model.Sellers, 
                        rule = constraint_sellers_out)

In [ ]:
def constraint_sellers_in(model, s):
    '''
    Ensures that each seller (s) has an warehouse entry sub route.
    '''

    return sum( model.x[j, warehouse, s]    
                for j in model.Customers if (cost[j, warehouse] > 0)) == 1

model.C5 = Constraint(  model.Sellers, 
                        rule = constraint_sellers_in)

In [ ]:
def constraint_customer_visited(model, i, s):
    '''
    To server the customer, you have to go through him.
    x[j, i, s] must be greater than or equal to a[i,s].
    If a[i,s] == 1, it means that the customer was served. 
    Therefore, the customer needs to have been visites previously. 
    '''

    return sum( model.x[j, i, s] 
                for j in model.Customers  if (i != j) & (cost[j, i] > 0)) == model.a[i,s] 

model.C6 = Constraint(  model.Customers, 
                        model.Sellers, 
                        rule = constraint_customer_visited)

In [ ]:
def constraint_flow(model, j, s):
    ''' 
    Flow Conservation Restriction.
    '''
    
    return  sum(    model.x[i, j, s] 
                    for i in model.Customers if (i!=j) & (cost[i,j] > 0)) - \
            sum(    model.x[j, i, s] 
                    for i in model.Customers if (i!=j) & (cost[j,i] > 0)) == 0

model.C7 = Constraint(  model.Customers, 
                        model.Sellers, 
                        rule = constraint_flow)

In [ ]:
def constraint_MTZ(model, i, j):
    '''
    Applies the Miller-Tucker-Zemlin (MTZ) constraint to the route between cities i and j.
    '''

    if (i != j) & (j > warehouse) & (i > warehouse):
        return model.u[i] - model.u[j] + bigM * sum(model.x[i,j,s] 
                                                    for s in model.Sellers) <= bigM - 1
    else:
        return Constraint.Skip

model.C8 = Constraint(  model.Customers, 
                        model.Customers, 
                        rule = constraint_MTZ)

#### Execução

In [ ]:
# Selecting and Creating the Solver
# ----------------------------
solver = SolverFactory('cplex')
#solver.options['timelimit'] = 30

# Calling the solver and solving the model
# ----------------------------

results = solver.solve( model, 
                        #tee = True, 
                        #keepfiles = True, 
                    )

print(f'FO value: {value(model.objective)}')

#### Plot do Resultado

In [ ]:
fig, ax = plt.subplots( nrows = 1,  
                        ncols = 1, 
                        figsize = (10,7),  
                        dpi = 110         )

sns.scatterplot(    data = df, x = "Log", y = "Lat", s = 40, linewidth = 0, color = '#111111' , ax = ax   )  

for i in df.index:
    ax.text( df.loc[i,'Log'], df.loc[i,'Lat'], str(df.loc[i, 'Customers']), color = 'black' , fontsize = 11    )


colors = ['blue', 'red', 'green', 'yellow']
for s in range(1, total_sellers + 1):
    for c1 in range(total_customers + 1):
        for c2 in range(total_customers + 1):
            if (value(model.x[c1, c2, s]) == 1):
                
                plt.arrow(   df.loc[c1, 'Log'], 
                                    df.loc[c1, 'Lat'], 
                                    df.loc[c2, 'Log'] - df.loc[c1, 'Log'],
                                    df.loc[c2, 'Lat'] - df.loc[c1, 'Lat'], 
                                    head_width = 0.1, 
                                    head_length = 0.1, 
                                    fc = 'white', 
                                    ec = colors[s],
                                    )


ax.set_title(   'Routes', color = '#646369', loc = 'left', pad = 15, fontsize = 13, weight = 'bold')
ax.tick_params( axis = 'both', colors = '#646369', labelsize = 10 )         

ax.spines['right'].set_visible(False)      
ax.spines['top'].set_visible(False)         


ax.spines['bottom'].set_color('#646369')       
ax.spines['left'].set_color('#646369')        


ax.set_xlabel(  'Longitude', color = '#646369', fontsize = 10, position = (0, 0), horizontalalignment = 'left' )
ax.set_ylabel(  'Latitude', color = '#646369', fontsize = 10, position = (0,1), horizontalalignment = 'right' )